In [6]:
import os
import googleapiclient.discovery
from dotenv import load_dotenv


In [7]:
load_dotenv(override=True) 
YOUTUBE_PUBLIC_API_KEY = os.environ['YOUTUBE_PUBLIC_API_KEY']
YOUTUBE_AWS_CHANNEL_ID = os.environ['YOUTUBE_AWS_CHANNEL_ID']

In [9]:


# --- Configuration ---
# Replace with your actual API key
# --- YouTube API Initialization ---
def get_youtube_service():
    """Initializes and returns the YouTube API service."""
    api_service_name = "youtube"
    api_version = "v3"
    return googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=YOUTUBE_PUBLIC_API_KEY
    )

# --- Get Channel Videos ---
def get_channel_videos(youtube_service, channel_id, max_results=50):
    """
    Fetches videos from a given YouTube channel.
    Args:
        youtube_service: The initialized YouTube API service object.
        channel_id: The ID of the YouTube channel.
        max_results: The maximum number of videos to retrieve per page (default is 50, max is 50).
    Returns:
        A list of video dictionaries.
    """
    all_videos = []
    next_page_token = None

    while True:
        try:
            request = youtube_service.search().list(
                part="snippet",
                channelId=channel_id,
                type="video",
                order="date",  # Order by date (newest first)
                maxResults=max_results,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response.get("items", []):
                video_id = item["id"]["videoId"]
                title = item["snippet"]["title"]
                description = item["snippet"]["description"]
                published_at = item["snippet"]["publishedAt"]
                thumbnail_url = item["snippet"]["thumbnails"]["high"]["url"]

                all_videos.append({
                    "video_id": video_id,
                    "title": title,
                    "description": description,
                    "published_at": published_at,
                    "thumbnail_url": thumbnail_url
                })

            next_page_token = response.get("nextPageToken")
            if not next_page_token:
                break # No more pages

        except googleapiclient.errors.HttpError as e:
            print(f"An HTTP error {e.resp.status} occurred:\n{e.content}")
            break
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            break
    return all_videos

# --- Main Execution ---
if __name__ == "__main__":
        youtube = get_youtube_service()
        #print(f"Fetching videos for channel ID: {CHANNEL_ID}")
        videos = get_channel_videos(youtube, CHANNEL_ID, max_results=25) # Fetching 25 videos per page

        if videos:
            print(f"Found {len(videos)} videos:")
            for i, video in enumerate(videos):
                print(f"--- Video {i+1} ---")
                print(f"Title: {video['title']}")
                print(f"Video ID: {video['video_id']}")
                print(f"Published At: {video['published_at']}")
                print(f"Thumbnail: {video['thumbnail_url']}")
                print(f"Description: {video['description'][:100]}...") # Print first 100 chars of description
                print("-" * 30)
        else:
            print("No videos found or an error occurred.")

Found 1 videos:
--- Video 1 ---
Title: AWS Certified Solutions Architect Associate Practice Questions (Part 1) May 2025
Video ID: fNt1pK6XVo4
Published At: 2025-05-22T18:46:48Z
Thumbnail: https://i.ytimg.com/vi/fNt1pK6XVo4/hqdefault.jpg
Description: AWS Solutions Architect Associate Exam | AWS SAA C03 Exam Dump Questions | AWS Solutions Architect A...
------------------------------
